# Connecting to server by Running following code on Terminal

ssh -f -N -L 9545:localhost:8545 root@159.203.19.65

ssh pikachu@10.10.1.224

password: 4689

geth --rpc --rpcport=8545 --rpccorsdomain=* --rpcapi=web3,db,net,eth

# And run the visual.py (with contract_abi.json) on your computer to update the Database.

# 0. Imports and connecting to database

In [1]:
import mysql.connector
from mysql.connector import errorcode
import csv
import pandas as pd
import numpy as np

import logging, sys, os
import matplotlib.pyplot as plt
import numpy as np
import heapq
import webbrowser
import web3
import json
import binascii
import pymysql.cursors
from web3 import Web3
from web3.contract import ConciseContract
from datetime import datetime
from matplotlib.widgets import Button
import web3 
from web3 import eth
from web3 import Web3
from web3.contract import ConciseContract
eth_host = "http://localhost:9545"
contract_json_name = 'contract_abi.json'
contract_address = Web3.toChecksumAddress('0x6fb3e0a217407efff7ca062d46c26e5d60a14d69'.lower()) 


#Set up eth connection
def eth_connect():
    web3 = Web3(Web3.HTTPProvider(eth_host))
    with open(contract_json_name, "r") as abi_definition:
        contract_abi = json.load(abi_definition)
        myContract = web3.eth.contract(address=contract_address, abi=contract_abi)
    return web3, myContract, contract_abi

web3, myContract, contract_abi = eth_connect()

# Use this to Test the eth connection (Get Transaction Counts)
#web3.eth.getTransactionCount(Web3.toChecksumAddress('0x3f5CE5FBFe3E9af3971dD833D26bA9b5C936f0bE'))

# Connecting to the database
try:
    cnn = mysql.connector.connect(
        host = 'iotexdbdev2.cs4r6igeqju2.us-east-1.rds.amazonaws.com',
        port = '3306',
        user = 'iotexadmin',
        password = '6y3vPj4Sa6wAU' ,
        database ='eth'
    )
    print ("database connected")
except mysql.connector.Error as e:
    if e.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("username or password is wrong")
    elif e.errno == errorcode.ER_BAD_DB_ERROR:
        print("database does not exist")
    else:
        print(e)
cursor = cnn.cursor()


database connected


In [2]:
# Defian a holder Object by 
#  Holder's name,address,out_tran(transfer money out),in_tran(transfer mondy in),asset(current money hold)
#  LargeTxn (Ture/False ## If a account has transcation more than 800, we take it as largeTxn account)
class holder:
    def __init__(self,name, address, out_tran,in_tran,asset,largeTxn):
        self.name = name
        self.address = address
        self.out_tran = out_tran
        self.in_tran = in_tran
        self.asset = asset
        self.largeTxn = largeTxn
        
#name - the holder's name
#address - the holder's eth address
#out_tran - a list of all the out_tran from this account (address,amount)
## out_tran e.g. "0x3f5CE5FBFe3E9af3971dD833D26bA9b5C936f0bE",2333

#in_tran - a list of all the in_tran from this account (address,amount)
## in_tran e.g. "0x3f5CE5FBFe3E9af3971dD833D26bA9b5C936f0bE",2333

#asset- the holder's current money
#largeTxn - If the account is larget transcation account


# 1. Get monitored holders infomation from Database

In [3]:
# SQL query to get data from database which all the monitored holders
sql_monitoring = "Select holder,address from monitored"
df_monitoring = pd.read_sql(sql_monitoring, cnn)
monitoring_array = df_monitoring.values
# monitoring_array is a array of all data collected from query sentence
# Structer: monitoring_array[”The id of monitored people"]["monitored address"]

In [4]:
sql_out = "Select holder, address,GROUP_CONCAT(CONCAT(toAddr,',',quantity,']')) list from transfers, monitored where LOWER(transfers.fromAddr) = LOWER(monitored.address) GROUP BY fromAddr ;"
df_out = pd.read_sql(sql_out, cnn)
monitored_out = df_out.values
out_list = list()#send out
with_fromAdd = []
for i in monitored_out:
    alist = i[2].split('],',)
    alist[-1] = alist[-1].replace(']','')
    with_fromAdd = [i[0],i[1],alist]
    out_list.append(with_fromAdd)
#out_list[[”name"]["address"][out_tran,amount (list)]]

In [5]:
sql_in = "Select holder, address,GROUP_CONCAT(CONCAT(fromAddr,',',quantity,']')) list from transfers, monitored where LOWER(transfers.toAddr) = LOWER(monitored.address) GROUP BY toAddr ;"
df_in = pd.read_sql(sql_in, cnn)
monitored_in = df_in.values
in_list = list()#send out
with_toAdd = []
for i in monitored_in:
    alist = i[2].split('],',)
    alist[-1] = alist[-1].replace(']','')
    with_toAdd = [i[0],i[1],alist]
    in_list.append(with_toAdd)
#in_list[[”name"]["address"][in_tran,amount (list)]]

In [6]:
#put transcation info and holder's detail insidd holder_list
holder_list = list()
for i in monitoring_array:
    _in = []
    _out = []
    for m in in_list:
        if m[1].lower() == i[1].lower():
            _in = [item.lower() for item in m[2]]
            break;
    for n in out_list:
        if n[1].lower() == i[1].lower():
            _out = [item.lower() for item in n[2]]
            break;            
    holder_list.append(holder(i[0],i[1].lower(),_out,_in,0,False))

In [7]:
#update the holder's asset info and put inside holder_list_updated
holder_list_updated =list()
for people in holder_list:
    money = 0
    for i in people.out_tran:
        try:
            money = money - float(i.split(',')[1])
        except:
            moeny = money - 0 
    for m in people.in_tran:
        try :
            money = money + float(m.split(',')[1])
        except:
            moeny = money + 0 
    people.asset = money
    holder_list_updated.append(people)

# 2. Get all holders infomation from Database

In [8]:
# SQL query to get transfer_out info from database
sql_all_out = "Select fromAddr,GROUP_CONCAT(CONCAT(toAddr,',',quantity,']')) list from transfers  GROUP BY fromAddr ;"
df_all_out = pd.read_sql(sql_all_out, cnn)
all_out = df_all_out.values
all_out_list = list()#send out
all_out_ele = []
for i in all_out:
    alist = i[1].split('],',)
    alist[-1] = alist[-1].replace(']','')
    all_out_ele = [i[0],alist]
    all_out_list.append(all_out_ele)

In [9]:
# SQL query to get transfer_in info from database
sql_all_in = "Select toAddr,GROUP_CONCAT(CONCAT(fromAddr,',',quantity,']')) list from transfers  GROUP BY toAddr ;"
df_all_in = pd.read_sql(sql_all_in, cnn)
all_in = df_all_in.values
all_in_list = list()#send out
all_in_ele = []
for i in all_in:
    alist = i[1].split('],',)
    alist[-1] = alist[-1].replace(']','')
    all_in_ele = [i[0],alist]
    all_in_list.append(all_in_ele)

In [10]:
# merge all_in_list and all_out_list together and put as holder in acc_list
acc_list = list()
for m in all_in_list:
    _out = []
    _in = [item.lower() for item in m[1]]
    for n in all_out_list:
        if m[0].lower() == n[0].lower():
            _out = [item.lower() for item in n[1]]
            break;
    acc_list.append(holder("",m[0].lower(),_out,_in,0,False))#holder:name,address,out_tran,in_tran,asset

In [11]:
#update the all holder's asset info and put inside acc_list_updated
acc_list_updated =list()
for people in acc_list:
    money = 0
    for i in people.out_tran:
        try:
            money = money - float(i.split(',')[1])
        except:
            moeny = money - 0 
    for m in people.in_tran:
        try :
            money = money + float(m.split(',')[1])
        except:
            moeny = money + 0 
    people.asset = money
    acc_list_updated.append(people)#holder:name,address,out_tran,in_tran,asset,largeTxn

##Description:  
Now we have two list of people one named : holder_list_updated and acc_list_updated.  
Each list store a people (defined by class holder)  
A holder is defined as (name, address, out_tran,in_tran,asset,largeTxn)  


# 3. Identify the large Txn account

##Description
We think some of the accouts in acc_list are bank accout or eth exchange account  
In the evaluation process, we regards transfering token to address having largeTxn as "selling the token"  
By identify the large Txn account, we can use the web3 methods.

In [ ]:
# Warning: large run time
# Because of connection reason, this takes 30min+ to get the result.
# Recommend for printing out of the address result for further usage 
for people in acc_list_updated:
    if web3.eth.getTransactionCount(Web3.toChecksumAddress(people.address))>800:
        people.largeTxn = True

In [13]:
# Result from last time
largeTxn_list = [
'0x008197bf3340c3f59e59edec0665da9720ed1c83','0x0377a487d38540ff258fd7446ecb73d53d105b7f','0x05ee546c1a62f90d7acbffd6d846c9c54c7cf94c',
'0x0681d8db095565fe8a346fa0277bffde9c0edbbf','0x06f249956dd5e8d6938e2907a2b56178c84fca9b','0x073903b967e40ff9f1190df3adad3929a5d3903f',
'0x074976a8d5f07da5dada1eb248ad369a764bb373','0x0d0707963952f2fba59dd06f2b425ace40b492fe','0x0d7a3043abdd146a113099b4a9a2ff1523bcc83c',
'0x1edb2378e2f5671b98f3093e9f9671df4e07c4a4','0x221bcdbad28f8d978bd91e62f6eef3e23526943f','0x237cd16ad77ed39e18dd2a958be24ef238750e6c',
'0x2585fc82e7d32246fbfd9e17bb3c7faeb241570e','0x260eecb85a4947db5d5a6da718bc7c98622bd782','0x28c73a915d066f5bcc5e0225b44244934d39bab1',
'0x2b5634c42055806a59e9107ed44d43c426e58258','0x2ba56a2415dedaaef2a54a1a6cc90d4555e0000d','0x30f5574a4c199372e8393f5f4f36c939757da878',
'0x3679b3fff2033d60684c51bc54ee1e371c62a77b','0x375fb217363d9bb8bd136b00e9f4e4a09b318205','0x3900f982a33cbe4b27bef4044a70e91f31fe4f53',
'0x3c3aef0043c8629556d0b20425aca08a1509d1dc','0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be','0x45c7449977654754892c3f2afd1df428fa199dd4',
'0x47590e87b48f49149e8cdb934605617a42c38dcf','0x4edbc97446dea5606c303bc60b9b056b69161240','0x52a11f06a30cbb704abbd069a93fdf6d853fc3c5',
'0x547269c54497404c144bfb5063ca41d51b7b5a88','0x564286362092d8e7936f0549571a803b203aaced','0x5aa943321f5b22bcf7a7d09c7eab8137aa1b72f8',
'0x5ab9d116a53ef41063e3eae26a7ebe736720e9ba','0x60bcb1252268d830347f3d981e6776bfc28f0c0a','0x63d118842d21a96b03bdc01672164afea93fd937',
'0x6446896b10465549af133800f86155f48a188943','0x67c0fc4b0490ab7e76c08c2bbd30fac0059bbc7a','0x699a2d50bfbc950ac016d424a9245bb80ad9d82b',
'0x6a36492fd141fe6d631b7f1d9d37943583d752eb','0x6e8488adb6e6e7fa509efd6b142b79944a6881af','0x6ef7a306b59d0245e0517f56a49035ae4427ad72',
'0x6fe1d5ae5ea6ba334bd4c1cc2dd8b6695314d82d','0x746e17548b63b0689e054d84a09abe1f5b70672c','0x751cc568cb9a7becd3001217cbeba8563e58fc7c',
'0x7598c75ce00fe7a401f46738ec357a6bc73627d2','0x75a5aa3b29dade574fb7648c3491f95d0c415dcf','0x75bdcb25686ba417657e1639b32529faf10f5f53',
'0x75e89d5979e4f6fba9f97c104c2f0afb3f1dcb88','0x76a2aaa2e54ca4fac6af634e18a5e6a3d7c80991','0x7a808d86ff19c7003320be7a683a9e1dca816f38',
'0x7ac34681f6aaeb691e150c43ee494177c0e2c183','0x7c3c05d63dd13aab94feb494d3bfe3db19b8dd69','0x7d04b0bea87436946d7cc33801aa6a69ea2d0072',
'0x819f488e49ad486961e13e5504a1af1b2e579f58','0x891e8ebd0430ab4360b7b74996c2aee650a960d9','0x8af55e3395b878d7e02971fb35f6f214a97c83e4',
'0x9175c1ef119c1ccd30c398e5374e036e2105ac2b','0x935c872530a4d084b212d6d07a3a48815a37916a','0x9470291e0ddc6f0187991d11054e5955ba2df4db',
'0x9664ed0820a10e95851480b9821532994b7bd9d1','0x9733ae7ececa99c8b1db998a7e8e37bcb3f68ae2','0x97ca6ae239e5476b546fc873002bf117cc52f6df',
'0x99781ad21621a30881aaa21559463c38cf1a9ef9','0x9b66137da669f2e6277832a3d2c91edfe963e64c','0x9b7bed561f48ad8d08b56b40a7addfb6be6b377b',
'0xa464b86efce0f91abb46f6df0f09df974dfcae05','0xa8282a3d5c9699a3e5e8d387c8e2f51fc8110389','0xa98a6f7579fe963f5f38eca3aa2721841e1c1992',
'0xa9c7d31bb1879bff8be25ead2f59b310a52b7c5a','0xad0a2478ba25816365aa0e32d77aff92b9e34efc','0xb006345bee21d33eb8b33d72423e755e2b0bf97b',
'0xb10d4d83491e7be1f9451065c9dc5909b717a28c','0xb90dd77405fdde22a4f37e98b2c8574cf4e20c98','0xba1970990162c7e233fd947201baa8843fc1e62e',
'0xbbf52f84df70c70d58b50b6bcf4ec90254d7eb13','0xc30ff49429b3077fc2b9a910b072f2300dc41b9a','0xc53c099b03e3811d8b70cf27997f6c389db8de6d',
'0xca6854361b6083134cd347e1d4b5e7e607fbc4a2','0xcf6aaa597036008be56cfaf8cca2a1a51021f7cb','0xd00ae2d78d950b07e6cc30dfbebcd30a02344068',
'0xd2fd00999005929be5e3983c74c196e029cee5a2','0xd454ed303748bb5a433388f9508433ba5d507030','0xd551234ae421e3bcba99a0da6d736074f22192ff',
'0xe6fcd6eeafd6cba664b8ef468687a885dd87dec7','0xecc576bc071f11ebb81aee11b5c50c8412f658fa','0xee6a089f26d549482ad14ff305ea761c45e7dbd5',
'0xeec3e3ab52572a51d93572e54fbb63824d1756bd']


In [14]:
largeTxn_acc = list()
largeTxn_addr = list()
for i in acc_list_updated:
    if i.largeTxn == True:
        largeTxn_acc.append(i)
for add in largeTxn_acc:
    largeTxn_addr.append(add.address)

# 4. Modify the data for further calculation

In [15]:
# a NEW class called holder_con easy for doing calculation
# separately store transaction address and amount
# by adding two more attributes called out_tran_amount and in_tran_amount
class holder_con:
    def __init__(self,name, address, out_tran,out_tran_amount, in_tran, in_tran_amount,asset,largeTxn):
        self.name = name
        self.address = address
        self.out_tran = out_tran
        self.out_tran_amount = out_tran_amount
        self.in_tran = in_tran
        self.in_tran_amount = in_tran_amount
        self.asset = asset
        self.largeTxn = largeTxn

In [16]:
#update the holder's transcation address and money amout separately
all_named_holders = list()
for people in holder_list_updated:
    out_tran_list = list()
    out_money_list = list()
    in_tran_list = list()
    in_money_list = list()
    
    for i in people.out_tran:
        try:
            address_out = i.split(',')[0]
            out_money = float(i.split(',')[1])
        except:
            address_out = ''
            out_money = 0
        if address_out not in largeTxn_list:
            out_tran_list.append(address_out)
            out_money_list.append(out_money)
        else:
            continue
        
    for m in people.in_tran:
        try:
            address_in = m.split(',')[0]
            in_money = float(i.split(',')[1])
        except:
            address_in = ''
            in_money = 0
            
        in_tran_list.append(address_in)
        in_money_list.append(in_money)
        
    all_named_holders.append(holder_con(people.name,people.address.lower(),
                            out_tran_list, out_money_list 
                            ,in_tran_list,in_money_list,
                            people.asset,people.largeTxn))
    #holder_con:name, address, out_tran,out_tran_amount, in_tran, in_tran_amount,asset,largeTxn

In [17]:
#update the all account's transcation address and money amout separately
all_accounts = list()
for people in acc_list_updated:
    out_tran_list = list()
    out_money_list = list()
    in_tran_list = list()
    in_money_list = list()
    
    for i in people.out_tran:
        try:
            address_out = i.split(',')[0]
            out_money = float(i.split(',')[1])
        except:
            address_out = ''
            out_money = 0
        
        if address_out not in largeTxn_list:
            out_tran_list.append(address_out)
            out_money_list.append(out_money)
        else:
            continue
        
    for m in people.in_tran:
        try:
            address_in = m.split(',')[0]
            in_money = float(i.split(',')[1])
        except:
            address_in = ''
            in_money = 0
            
        in_tran_list.append(address_in)
        in_money_list.append(in_money)
        
        
        
    all_accounts.append(holder_con("",people.address.lower(),
                            out_tran_list, out_money_list 
                            ,in_tran_list,in_money_list,
                            people.asset,people.largeTxn))
    #holder_con:name, address, out_tran,out_tran_amount, in_tran, in_tran_amount,asset,largeTxn
len(all_accounts)

13906

In [18]:
for io in all_accounts:
    if io.address in largeTxn_list:
        all_accounts.remove(io)

# 5. Run Score Calculation

In [19]:
class holder_score:
    def __init__(self,name, address,asset,score):
        self.name = name
        self.address = address
        self.asset = asset
        self.score = score

In [20]:
def getAcc (address):
    for people in all_accounts:
        end_address = ''
        if people.address == address:
            try:
                max_ = max(people.out_tran_amount)
                add_choosed = people.out_tran[people.out_tran_amount.index(max_)]
                end_address = add_choosed
            except:
                add_choosed = ''
                max_ = 0
            return add_choosed,max_,end_address
    return ''
    

In [21]:
holder_score_list = list()
for people in all_named_holders:
    end_address = ''
    try:
        max_ = max(people.out_tran_amount)
        add_choosed = people.out_tran[people.out_tran_amount.index(max_)]
    except:
        add_choosed_ = ''
        max_ = 0
    
    cont = 0
    while add_choosed !='':
        cont += 1
        if cont >3:
            break;
        try:
            add_choosed,max_,end_address = getAcc(add_choosed)
        except:
            add_choosed,max_= '',0
    holder_score_list.append([people.name,people.address,people.asset,max_,end_address])


In [22]:
import pandas as pd
df = pd.DataFrame(holder_score_list)
df = df.sort_values(3,ascending=False)
df

,0,1,2,3,4
7,ASBV,0x85ac9e682995ebebde8ff107fbbbfe7c40992e4a,0.0,23999000.0,0x2784e6a6232078b3da167e337403d2eb44e0e9b8
29,Hofan,0xa7b5400973ee73d360f4adc88a011d7be388fd64,0.0,3040000.0,0x952e475ea798d3c4ef8331a8b0eda849caa262fc
53,ZH Capital-1,0x2990968b2f60fb5116544850fc7c730638ad7827,0.0,2400000.0,0x1c4b70a3968436b9a0a9cf5205c787eb81bb558c
41,RedNova -Du Capital,0x59f1a8893da75e5041c946b08474dd3f5ff08815,15650000.0,2400000.0,0x59f1a8893da75e5041c946b08474dd3f5ff08815
2,Thomas,0x0c8e046870419a18ccbf90f49d44f73854420bc1,0.0,1429280.0,0x4be7ed3497342e1972d375d682dcab1187358dd0
57,ZMT Capital,0xf4af533c14d8c68aa2d5375526ec79876f9ade98,0.0,900000.0,0x9af385c7c27e127b3e7998e22c332d4bdfa72732
15,Bril Wang (I),0xdedc3bbb6ba7da62c4114698efec9b7fede994eb,0.0,864000.0,0xea3f9d498c335e27ba7dcff3e2eea93cc6f99bb9
33,LanHu,0x8bbca007d6743d8bf260725049a95d805f4415ce,0.0,500000.0,0x1c4b70a3968436b9a0a9cf5205c787eb81bb558c
44,Signal-1,0xba0490b8f8960d966437705a26a101f913e6271b,0.0,119960.0,0xee1e33029c2104993e4536be502990284e77080d
6,Andrew(Danhua),0xdec4d6b0ecf973f3f2d7b71d3ae1131b7669347f,0.0,0.0,


# Useful Tools

In [52]:
#holder_con:name, address, out_tran,out_tran_amount, in_tran, in_tran_amount,asset,largeTxn
def get_holder_info(address):
    this_user = []
    for people in all_named_holders:
        if people.address == address:
            this_user = [people.name,people.out_tran,people.out_tran_amount,people.in_tran,people.in_tran_amount]
            break;
    return this_user

In [108]:
def get_acc_info(address):
    this_user = []
    for people in all_accounts:
        if people.address == address:
            this_user = [people.name,people.asset]
            break;
    return this_user

In [151]:
get_acc_info('0x952e475ea798d3c4ef8331a8b0eda849caa262fc')

['', 0.0]